### Imports

In [7]:
import torch
import pandas as pd
from datetime import timedelta
from datetime import datetime

import numpy as np

from nera.data import *
from nera.models.ratings import BerrarManual, BerrarNumerical, BerrarAnalytical
from nera.reference import *
from nera.trainer import Trainer
from nera.utils import print_rating_diff


### Data acquisition

In [8]:
da = DataAcquisition()
df = da.get_data(FROM_CSV, fname="../resources/other_leagues.csv")
df['DT'] = pd.to_datetime(df['DT'], format="%Y-%m-%d %H:%M:%S")
data_transform = DataTransformation(df, timedelta(365))
df = df[(df['League'] != 'EuroLeague') & (df['League'] != 'EuroCup')] 
df = df.reset_index()

transform = DataTransformation(df, timedelta(365))
dataset = transform.get_dataset(node_f_extract=False, edge_f_one_hot=True)

team_count = transform.num_teams

trainer = Trainer(dataset, train_ratio=1, loss_fn=torch.nn.MSELoss)
reference_maker = RatingReference(transform.num_teams)

berrar_params = {
            'alpha_h': torch.tensor(80, dtype=torch.float64),
            'alpha_a': torch.tensor(80, dtype=torch.float64),
            'beta_h': torch.tensor(2, dtype=torch.float64),
            'beta_a': torch.tensor(2, dtype=torch.float64),
            'bias_h': torch.tensor(0, dtype=torch.float64),
            'bias_a': torch.tensor(0, dtype=torch.float64),
            'lr_h_att': torch.tensor(0.1, dtype=torch.float64),
            'lr_a_att': torch.tensor(0.1, dtype=torch.float64),
            'lr_h_def': torch.tensor(0.1, dtype=torch.float64),
            'lr_a_def': torch.tensor(0.1, dtype=torch.float64),
            'default': 1000.
        }

2024-03-14 15:56:21.880 | INFO     | nera.data._data_saving_loading:load_data_csv:70 - 21100 rows loaded from ../resources/other_leagues.csv


### Manual berrar NN vs reference

In [9]:

computed = reference_maker.compute_reference('berrar', dataset, **berrar_params)

print()
berrar_man = BerrarManual(team_count=transform.num_teams, **berrar_params)
trainer.dataset = dataset
trainer.model = berrar_man
acc_late = trainer.train(epochs=1, val_ratio=0, verbose=True)

err = False
for rating in range(len(computed)):
    for i in range(len(computed[rating])):
        cmp = float(computed[rating][i])
        net = float(berrar_man.ratings[rating][i])
        if abs(cmp - net) > 0.1:
            print(rf'rating {rating}::ERROR on index {i}:: {cmp} / {net}')
            err = True
        if i < 5:
            print(f'rating {rating}::{i}:: computed: {cmp:10.3f} || net: {net:10.3f}')
if not err:
    print("...")
    print("[SUCCESS]: Computed berrar rating is the same as berrar from NN")

/home/kubos/anaconda3/envs/bp/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
2024-03-14 15:56:30.911 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 2196060.500, training accuracy: 0.02%


rating 0::0:: computed:   1016.000 || net:   1016.000
rating 0::1:: computed:   1028.400 || net:   1028.400
rating 0::2:: computed:   1103.100 || net:   1103.100
rating 0::3:: computed:   1113.600 || net:   1113.600
rating 0::4:: computed:    904.800 || net:    904.800
rating 1::0:: computed:   1011.900 || net:   1011.900
rating 1::1:: computed:   1191.900 || net:   1191.900
rating 1::2:: computed:    948.600 || net:    948.600
rating 1::3:: computed:    958.700 || net:    958.700
rating 1::4:: computed:   1106.500 || net:   1106.500
...
[SUCCESS]: Computed berrar rating is the same as berrar from NN


### Analytical vs numerical pass
1) gradient check

In [10]:
from torch.autograd import gradcheck
from nera.models.ratings._berrar._analytical import _berrar_fn
# gradcheck takes a tuple of tensors as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True))
test = gradcheck(_berrar_fn, input, eps=1e-6, atol=1e-4)
print(test)

True


2. analytical vs numerical nn pass

In [11]:
berrar_numerical = BerrarNumerical(team_count, hp_grad=True, **berrar_params)
berrar_analytical = BerrarAnalytical(team_count, hp_grad=True, **berrar_params)

trainer.train_ratio = 0.8 

In [12]:
trainer.model = berrar_numerical
trainer.train(verbose=True, epochs=1, val_ratio=0.2)
#trainer.test(verbose=True)
print(berrar_numerical.ratings)
print(berrar_numerical.hyperparams)

2024-03-14 15:56:43.028 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 1498929.005, training accuracy: 58.60%
2024-03-14 15:56:43.031 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: 378540.491, validation accuracy: 58.96%


[Parameter containing:
tensor([1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000.

In [13]:
trainer.model = berrar_analytical
trainer.train(verbose=True, epochs=1, val_ratio=0.2)
#trainer.test(verbose=True)
print(berrar_analytical.ratings)
print(berrar_analytical.hyperparams)

2024-03-14 15:56:58.443 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 1503504.992, training accuracy: 58.61%
2024-03-14 15:56:58.445 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: 378290.539, validation accuracy: 58.96%


[Parameter containing:
tensor([1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000.